# Neuronpedia Explanations — Quick Look

This notebook loads the cached Neuronpedia explanations JSON and inspects its structure (keys, lengths) and a tabular view.

In [2]:
import os, json, glob, time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Style similar to other notebooks
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'font.size': 12,
    'font.family': 'sans-serif',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.5,
    'grid.linewidth': 1.0,
    'legend.frameon': False,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
})

# Default cache path from the fetch script run
DEFAULT_JSON = '../../models/NeuronpediaCache/gemma-2-2b/12-gemmascope-res-65k__l0-21.json'
json_path = DEFAULT_JSON

if not os.path.exists(json_path):
    # Fallback: pick most-recent JSON under the cache directory
    candidates = glob.glob('models/NeuronpediaCache/**/*.json', recursive=True)
    if not candidates:
        raise FileNotFoundError('No cached explanations found under models/NeuronpediaCache.')
    candidates.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    json_path = candidates[0]

print('Using JSON:', json_path)

Using JSON: ../../models/NeuronpediaCache/gemma-2-2b/12-gemmascope-res-65k__l0-21.json


## Load JSON and Inspect Structure

In [3]:
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print('Type:', type(data))
if isinstance(data, dict) and 'explanations' in data:
    data = data['explanations']
    print('Unwrapped explanations list from dict.')

if not isinstance(data, list):
    raise TypeError(f'Unexpected top-level JSON type: {type(data)} (expected list)')

print('Num explanations:', len(data))
if data:
    first = data[0]
    print('First item type:', type(first))
    if isinstance(first, dict):
        print('First item keys:', sorted(list(first.keys())))

# Inspect key coverage across a sample
sample_n = min(1000, len(data))
keys_union = set()
keys_intersection = None
for i in range(sample_n):
    it = data[i]
    if isinstance(it, dict):
        k = set(it.keys())
        keys_union |= k
        keys_intersection = k if keys_intersection is None else (keys_intersection & k)

print('Union of keys (sample):', sorted(list(keys_union))[:50], ('...' if len(keys_union) > 50 else ''))
print('Intersection of keys (sample):', sorted(list(keys_intersection)) if keys_intersection else [])

Type: <class 'list'>
Num explanations: 65344
First item type: <class 'dict'>
First item keys: ['description', 'explanationModelName', 'index', 'layer', 'modelId', 'typeName']
Union of keys (sample): ['description', 'explanationModelName', 'index', 'layer', 'modelId', 'typeName'] 
Intersection of keys (sample): ['description', 'explanationModelName', 'index', 'layer', 'modelId', 'typeName']


## Normalize to DataFrame

In [4]:
# json_normalize handles nested structures if any
df = pd.json_normalize(data)
print('DataFrame shape:', df.shape)
print('Columns:', list(df.columns)[:50], ('...' if df.shape[1] > 50 else ''))
df.head(3)

DataFrame shape: (65344, 6)
Columns: ['modelId', 'layer', 'index', 'description', 'explanationModelName', 'typeName'] 


,modelId,layer,index,description,explanationModelName,typeName
0,gemma-2-2b,12-gemmascope-res-65k,53844,variables and mathematical symbols in technic...,None,oai_token-act-pair
1,gemma-2-2b,12-gemmascope-res-65k,53847,opening and closing parentheses and backslashes,None,oai_token-act-pair
2,gemma-2-2b,12-gemmascope-res-65k,53890,data type declarations in code,None,oai_token-act-pair


In [7]:
# Check all unique explanationModelNames
print(df['explanationModelName'].unique())

# Get counts
print(df['explanationModelName'].value_counts())

[None 'gpt-4o-mini']
explanationModelName
gpt-4o-mini    139
Name: count, dtype: int64


In [8]:
# Get number of entries
print(len(df))

65344


In [10]:
# Get number of non empty descriptions
print(df['description'].notna().sum())

65344


In [11]:
print(df['typeName'].unique())

['oai_token-act-pair']


In [3]:
# Load the other cache file 12-gemmascope-res-65k__l0-21_classified_lambda
with open("../../outputs/feature_classification/gemma-2-2b/12-gemmascope-res-65k__l0-21_formatting_classified_deepseek-v3-0324.json", "r") as f:
    classified_features = json.load(f)
    
# Convert to dataframe
df_lambda = pd.DataFrame(classified_features)
df_lambda.head(3)

,feature_id,description,label
0,gemma-2-2b-12-gemmascope-res-65k-53844,variables and mathematical symbols in technic...,not-related
1,gemma-2-2b-12-gemmascope-res-65k-53847,opening and closing parentheses and backslashes,related
2,gemma-2-2b-12-gemmascope-res-65k-53890,data type declarations in code,related


In [4]:
# Take value counts of classification column and divide by total length
df_lambda["label"].value_counts(normalize=True)

label
not-related    0.764462
related        0.235538
Name: proportion, dtype: float64